#### Feature Store
- Get Gold data and save to Feature Store for modelling


In [ ]:
# Import libraries
from databricks.feature_engineering import FeatureEngineeringClient
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, monotonically_increasing_id

In [ ]:
# get variables from enviroment
dbutils.widgets.text("gold_data_table", "goldtable")
dbutils.widgets.text("feature_table_name", "occupancy_feature")
dbutils.widgets.text("catalog_name", "prod-bms")
dbutils.widgets.text("schema_name", "occupancy_project")

gold_table_name = dbutils.widgets.get("gold_data_table")
feature_table_name = dbutils.widgets.get("feature_table_name")
catalog_name = dbutils.widgets.get("catalog_name")
schema_name = dbutils.widgets.get("schema_name")

##### Feature Table

In [ ]:
# Read gold data
table_name = f"`{catalog_name}`.{schema_name}.{gold_table_name}"
df = spark.read.table(table_name)

# add index column for primary key
window_spec = Window.orderBy(monotonically_increasing_id())
df1 = df.withColumn("Id", row_number().over(window_spec)).select("Id", *df.columns)

In [ ]:
# Instantiate the feature engineering client
fe = FeatureEngineeringClient()

# feature table name
feature_table = f"`{catalog_name}`.{schema_name}.{feature_table_name}"

# create feature table
try:
    if fe.get_table(name=feature_table):
        fe.write_table(
            name=feature_table,
            df=df1,
            mode="merge"
        )
        print(f"Write data operation to {feature_table} completed.")
except:
    fe.create_table(
        name=feature_table,
        primary_keys=["Id"],
        df=df1,
        description="Feature Table data for Occupancy Project",
        tags={"source": "Gold", "format": "delta"}
    )
    print(f"Create table operatio completed for {feature_table}")